# Introduction to Credit Risk Models and Capital

### Introduction

More than ever before Credit Risk Models are being used to steer strategic decisions in the economy.

Banks use credit risk models to help decide who to make loans to and how to manage the risk of those loans.  Part of that risk management is ensuring they have enough capital (spare cash) to weather shocks.

Banks use complex internal risk models, they are subject to regulatory supervision, and are ultimately subject to market discipline. They must have a minimum amount of holding capital as a buffer against shocks. 

This Minimum buffer capital is dynamic and the provisions a bank holds are directly determined by:
    
        Credit risk models
        Market risk models (liquidity, ir, commodities)
        Operational risk models
        Fraud risk models
        Insurance risk models

In this Capstone Project we will focus exclusively on Credit Risk Models, which in itself is a broad and deep subject matter.

Regulation allows models that use big data, machine learning techniques and analytical models, and these models are subject to regulatory oversight. Regulations define the inputs and outputs of the models, and how these outputs are combined to determine capital and provisions. Impact of model errors impact profitability, solvency, shareholders and the macroeconomy as a whole.

As is the case for almost all data science we need to ensure that our data and our models are of good quality.
    
#### 1) Credit Risk Models and Capital

In this capstone project we will estimate the amount of capital that needs to be held against individual loans based on their expected loss, which is a measure of the level of loss that can be expected losses are in normal circumstances. Banks do not expect everyone to repay their loans. Expcected Loss is made up of three credit risk components:

#### 2) Credit Risk Model Architecture:
      
Data Preparation: Use:

        *Internal Data - customer records and information on application 
        *External Data (Experian, Equifax, FICO score..., )
        *Expert Judgement (Qualitative judgement - help steer variable selection)
     
Along the way we will encounter missing values, outlier detection, outlier handling, categorization, variable selection, WoE, and information filtering.

#### 3) Scorecards and Capital Calculations
     
We will create the a model that supports an:

        *Application Scorecard - New credit applications (Logistic Regression)
        
The purpose of scoring is to discriminate between risky obligors. We will define a Ratings Scorecard calibrated to Probability of default that can be used to assess new loan applications.   


#### 4) Industry Practice and Expected Outcomes

PD credit Scoring models usually have 10-15 Chracteristics, and 70%-85% Area under ROC (depending on market & model)

LGD: 6-8 characteristics: LTV Loan to value, degree of collateral are usually included, and can be default risk included sometimes. 20-30% R-squared. This is worrying as it has a linear impact on capital. How can this be improved?
    
   
#### 5) Further work we can investigate LGD with the following models:

TO imporve results we can work on the model, or work on the data.
Working on the model: LGD alternative approaches:

            a) Random Forests
            b) Neural Networks
            c) Support Vector Machines
    
PROs: powerful models - universal approximation
CONS: loss of interpretability, marginal performance gains
Only used in white box set ups.

Work on the data: GIGO - Garbage in Garbage Out. Simple models perform well, so data can be the most important part.

## Introduction to Lending Club

Lending CLub is a peer to Peer Lending Club that provides a bridge between lenders and borrowers. It offers personal loans, business loans, auto refinancing and patient solutions for medical expenses. There have been 3 million customers borrowing $50bn through the the platform.

https://www.lendingclub.com/

### Peer to Peer Lending

Peer to Peer lenders provive P2P lending through online platforms, with the aim of passing on lower overheads to customers in the form of more affordable loans. Traditional banks have legacy overheads associated with a bricks and mortar presence, legacy systems, legacy loans and legacy customers (those reluctant to move online).

Peer to Peer lenders are a relatively new innovation in finance. They tend to have greenfield systems and architectures (simple data storage and retrieval) and use data science and machine learning methods to gain a competitve edge.

## The Dataset

The dataset is provided via Kaggle.  There are several versions of the Lending Club Dataset covering different time windows but we will explore the first dataset, covering loans from 2007 to 2015.

There are 887,379 loans in the raw dataset. 

The focus loans over 2007-2014 period, filtering out joint loan applications, and any loans issued after 2014. 
There are 466,284 loans saved down in the file loans_data_2007_2014.csv in the repo. It's 421MB and LFS ws used to upload to git.    

https://www.kaggle.com/janiobachmann/lending-club-first-dataset

### Data Preparation

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import pickle

## Import Data

The dataset contains all available data for more than 430,000 consumer loans issued from 2007 to 2014 by Lending Club: a large US peer-to-peer lending company. There are several different versions of this dataset. 

https://www.kaggle.com/wendykan/lending-club-loan-data/version/1

We divided the data into two periods because we assume that some data are available at the moment when we need to build Expected Loss models, and some data comes from applications after. 

Later, we investigate whether the applications we have after we built the Probability of Default (PD) model have similar characteristics with the applications we used to build the PD model.

**This way we can predict the EL for the applications from 2014 onwards.

In [2]:
loan_data_backup = pd.read_csv('../data/loan_data_2007_2014.csv')

C:\Users\User1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loan_data = loan_data_backup.copy()

## Explore Data

In [4]:
loan_data.shape

(466285, 75)

In [5]:
pd.options.display.max_columns = None

In [6]:
loan_data.head(2)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.0,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dropping Empty Columns
Let's drop the empty columns

In [7]:
# loan_data.drop('Unnamed: 0', axis=1, inplace=True) We will use this as a spare column to join on later....

In [8]:
null_columns=loan_data.columns[loan_data.isnull().all()]
loan_data[null_columns].isnull().sum()
empty_cols = null_columns.tolist()
for col in empty_cols:
    if col in loan_data.columns:
        loan_data.drop(col, axis=1, inplace=True)

In [9]:
print("The following empty columns have been dropped: " + "\n \n" + str(empty_cols))

The following empty columns have been dropped: 
 
['annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m']


In [10]:
for col in empty_cols:
    if col in loan_data.columns:
        loan_data.drop(col, axis=1, inplace=True)

In [11]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 58 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   id                           466285 non-null  int64  
 2   member_id                    466285 non-null  int64  
 3   loan_amnt                    466285 non-null  int64  
 4   funded_amnt                  466285 non-null  int64  
 5   funded_amnt_inv              466285 non-null  float64
 6   term                         466285 non-null  object 
 7   int_rate                     466285 non-null  float64
 8   installment                  466285 non-null  float64
 9   grade                        466285 non-null  object 
 10  sub_grade                    466285 non-null  object 
 11  emp_title                    438697 non-null  object 
 12  emp_length                   445277 non-null  object 
 13 

### Others

#### text columns to drop:

    * url: link internal deal info
    * desc: Free form text field from application process, grouped in desc
    * title: granular description of loan purpose, covered by desc

#### ethical columns to drop:

    * zip code: 888 zip codes: zip codes can be a proxy for race so this will be excluded.
    * here's alink to a paper on Proxy Discrimination in Data-Driven Systems
    * https://arxiv.org/pdf/1707.08120.pdf
 

In [12]:
mono_cols = []

for col in loan_data.columns:
    if len(loan_data[col].unique()) == 1:
        mono_cols.append(col)  
        
mono_cols

['policy_code', 'application_type']

In [13]:
        for cols in mono_cols:
            loan_data.drop(cols, axis=1, inplace=True) 
print("The following mono-line columns have been dropped " + str(mono_cols))

The following mono-line columns have been dropped ['policy_code', 'application_type']


In [14]:
text_cols = ['url', 'desc','title','emp_title']
ethics = ['zip_code']

for col in ethics, text_cols:
    loan_data.drop(col, axis=1, inplace=True)    

In [15]:
loan_data.describe()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
count,466285.000000,4.662850e+05,4.662850e+05,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,4.662810e+05,466285.000000,466256.000000,466256.000000,215934.000000,62638.000000,466256.000000,466256.000000,4.662850e+05,465945.000000,466256.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466140.000000,98974.000000,466256.000000,3.960090e+05,3.960090e+05,3.960090e+05
mean,233142.000000,1.307973e+07,1.459766e+07,14317.277577,14291.801044,14222.329888,13.829236,432.061201,7.327738e+04,17.218758,0.284678,0.804745,34.104430,74.306012,11.187069,0.160564,1.623020e+04,56.176947,25.064430,4410.062342,4408.452258,11540.686220,11469.892747,8866.014657,2588.677225,0.650129,85.344211,8.961534,3123.913796,0.009085,42.852547,0.004002,1.919135e+02,1.388017e+05,3.037909e+04
std,134605.029472,1.089371e+07,1.168237e+07,8286.509164,8274.371300,8297.637788,4.357587,243.485550,5.496357e+04,7.851121,0.797365,1.091598,21.778487,30.357653,4.987526,0.510863,2.067625e+04,23.732628,11.600141,6355.078769,6353.198001,8265.627112,8254.157579,7031.687997,2483.809661,5.265730,552.216084,85.491437,5554.737393,0.108648,21.662591,0.068637,1.463021e+04,1.521147e+05,3.724713e+04
min,0.000000,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,5.420000,15.670000,1.896000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,116571.000000,3.639987e+06,4.379705e+06,8000.000000,8000.000000,8000.000000,10.990000,256.690000,4.500000e+04,11.360000,0.000000,0.000000,16.000000,53.000000,8.000000,0.000000,6.413000e+03,39.200000,17.000000,0.000000,0.000000,5552.125349,5499.250000,3708.560000,957.280000,0.000000,0.000000,0.000000,312.620000,0.000000,26.000000,0.000000,0.000000e+00,2.861800e+04,1.350000e+04
50%,233142.000000,1.010790e+07,1.194108e+07,12000.000000,12000.000000,12000.000000,13.660000,379.890000,6.300000e+04,16.870000,0.000000,0.000000,31.000000,76.000000,10.000000,0.000000,1.176400e+04,57.600000,23.000000,441.470000,441.380000,9419.250943,9355.430000,6817.760000,1818.880000,0.000000,0.000000,0.000000,545.960000,0.000000,42.000000,0.000000,0.000000e+00,8.153900e+04,2.280000e+04
75%,349713.000000,2.073121e+07,2.300154e+07,20000.000000,20000.000000,19950.000000,16.490000,566.580000,8.896000e+04,22.780000,0.000000,1.000000,49.000000,102.000000,14.000000,0.000000,2.033300e+04,74.700000,32.000000,7341.650000,7338.390000,15308.158460,15231.310000,12000.000000,3304.530000,0.000000,0.000000,0.000000,3187.510000,0.000000,59.000000,0.000000,0.000000e+00,2.089530e+05,3.790000e+04
max,466284.000000,3.809811e+07,4.086083e+07,35000.000000,35000.000000,35000.000000,26.060000,1409.990000,7.500000e+06,39.990000,29.000000,33.000000,188.000000,129.000000,84.000000,63.000000,2.568995e+06,892.300000,156.000000,32160.380000,32160.380000,57777.579870,57777.580000,35000.030000,24205.620000,358.680000,33520.270000,7002.190000,36234.440000,20.000000,188.000000,5.000000,9.152545e+06,8.000078e+06,9.999999e+06


In [16]:
dates = loan_data['issue_d'].unique()
dates

array(['Dec-11', 'Nov-11', 'Oct-11', 'Sep-11', 'Aug-11', 'Jul-11',
       'Jun-11', 'May-11', 'Apr-11', 'Mar-11', 'Feb-11', 'Jan-11',
       'Dec-10', 'Nov-10', 'Oct-10', 'Sep-10', 'Aug-10', 'Jul-10',
       'Jun-10', 'May-10', 'Apr-10', 'Mar-10', 'Feb-10', 'Jan-10',
       'Dec-09', 'Nov-09', 'Oct-09', 'Sep-09', 'Aug-09', 'Jul-09',
       'Jun-09', 'May-09', 'Apr-09', 'Mar-09', 'Feb-09', 'Jan-09',
       'Dec-08', 'Nov-08', 'Oct-08', 'Sep-08', 'Aug-08', 'Jul-08',
       'Jun-08', 'May-08', 'Apr-08', 'Mar-08', 'Feb-08', 'Jan-08',
       'Dec-07', 'Nov-07', 'Oct-07', 'Sep-07', 'Aug-07', 'Jul-07',
       'Jun-07', 'Dec-13', 'Nov-13', 'Oct-13', 'Sep-13', 'Aug-13',
       'Jul-13', 'Jun-13', 'May-13', 'Apr-13', 'Mar-13', 'Feb-13',
       'Jan-13', 'Dec-12', 'Nov-12', 'Oct-12', 'Sep-12', 'Aug-12',
       'Jul-12', 'Jun-12', 'May-12', 'Apr-12', 'Mar-12', 'Feb-12',
       'Jan-12', 'Dec-14', 'Nov-14', 'Oct-14', 'Sep-14', 'Aug-14',
       'Jul-14', 'Jun-14', 'May-14', 'Apr-14', 'Mar-14', 'Feb-

In [17]:
macro_econ_back_up = pd.read_pickle('../data/macro_variables.pkl')

In [18]:
macro_econ = macro_econ_back_up.copy()

In [19]:
macro_econ.head()

,vix_mean,vix_min,vix_max,vix_spread,issue_d,year,ted_spread_mean,ted_spread_min,ted_spread_max,ted_spread_spread,nat_unemp,retail_sales
year,,,,,,,,,,,,
2007-01-01,11.04,9.89,12.14,2.25,Jan-07,2007-01-01,0.38,0.35,0.45,0.10,4.6,1.000000
2007-02-01,11.16,10.02,18.31,8.29,Feb-07,2007-02-01,0.33,0.31,0.37,0.06,4.5,0.997208
2007-03-01,15.16,12.19,19.63,7.44,Mar-07,2007-03-01,0.41,0.35,0.45,0.10,4.4,1.000638
2007-04-01,12.93,11.98,14.53,2.55,Apr-07,2007-04-01,0.49,0.43,0.57,0.14,4.5,0.995009
2007-05-01,13.30,12.76,14.08,1.32,May-07,2007-05-01,0.63,0.57,0.76,0.19,4.4,1.003760


In [20]:
loan_data=pd.merge(loan_data,macro_econ)

In [21]:
# loan_data.drop('year.1', axis=1, inplace=True)

In [22]:
print(loan_data.shape)

(466285, 62)


In [23]:
loan_data.head(2)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,vix_mean,vix_min,vix_max,vix_spread,year,ted_spread_mean,ted_spread_min,ted_spread_max,ted_spread_spread,nat_unemp,retail_sales
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,credit_card,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.0,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,0.0,NaN,NaN,NaN,25.05,20.73,30.59,9.86,2011-12-01,0.55,0.51,0.57,0.06,8.5,0.964808
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,car,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,0.0,NaN,NaN,NaN,25.05,20.73,30.59,9.86,2011-12-01,0.55,0.51,0.57,0.06,8.5,0.964808


In [24]:
loan_data.tail(2)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,vix_mean,vix_min,vix_max,vix_spread,year,ted_spread_mean,ted_spread_min,ted_spread_max,ted_spread_spread,nat_unemp,retail_sales
466283,466283,9604874,11457002,2000,2000,2000.0,36 months,7.9,62.59,A,A4,3 years,OWN,83000.0,Verified,Jan-14,Fully Paid,n,credit_card,CA,5.39,3.0,Feb-03,1.0,13.0,NaN,21.0,0.0,11404,21.5,27.0,w,0.00,0.00,2126.579838,2126.58,2000.00,126.58,0.0,0.0,0.0,Dec-14,1500.68,NaN,Apr-15,0.0,NaN,0.0,0.0,591610.0,53100.0,14.24,12.14,18.41,6.27,2014-01-01,0.2,0.17,0.22,0.05,6.6,0.995614
466284,466284,9199665,11061576,10000,10000,9975.0,36 months,19.2,367.58,D,D3,10+ years,MORTGAGE,46000.0,Verified,Jan-14,Current,n,other,CA,22.78,1.0,Feb-00,0.0,9.0,NaN,6.0,0.0,11325,70.8,22.0,f,3984.38,3974.41,8821.620000,8799.57,6015.62,2806.00,0.0,0.0,0.0,Jan-16,367.58,Feb-16,Jan-16,0.0,NaN,0.0,0.0,57477.0,16000.0,14.24,12.14,18.41,6.27,2014-01-01,0.2,0.17,0.22,0.05,6.6,0.995614


In [25]:
loan_data.columns.values

array(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim', 'vix_mean', 'vix_min', 'vix_max', 'vix_spread',
       'year

In [26]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 0 to 466284
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Unnamed: 0                   466285 non-null  int64         
 1   id                           466285 non-null  int64         
 2   member_id                    466285 non-null  int64         
 3   loan_amnt                    466285 non-null  int64         
 4   funded_amnt                  466285 non-null  int64         
 5   funded_amnt_inv              466285 non-null  float64       
 6   term                         466285 non-null  object        
 7   int_rate                     466285 non-null  float64       
 8   installment                  466285 non-null  float64       
 9   grade                        466285 non-null  object        
 10  sub_grade                    466285 non-null  object        
 11  emp_length                

## General Preprocessing

### Preprocessing continuous variables

Some columns representing continuous variables are in string format. These need to be cleaned before being converted to nummeric values.

In [27]:
loan_data['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [28]:
loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('\+ years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1 year', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('n/a',  str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year', '')

In [29]:
type(loan_data['emp_length_int'][0])

str

In [30]:
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'])

In [31]:
type(loan_data['emp_length_int'][0])

numpy.float64

In [32]:
loan_data['earliest_cr_line']

0         Jan-85
1         Apr-99
2         Nov-01
3         Feb-96
4         Jan-96
           ...  
466280    Apr-03
466281    Jun-97
466282    Dec-01
466283    Feb-03
466284    Feb-00
Name: earliest_cr_line, Length: 466285, dtype: object

In [33]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'], format = '%b-%y')

In [34]:
type(loan_data['earliest_cr_line_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [35]:
pd.to_datetime('2019-12-01') - loan_data['earliest_cr_line_date']

0        12752 days
1         7549 days
2         6604 days
3         8704 days
4         8735 days
            ...    
466280    6088 days
466281    8218 days
466282    6574 days
466283    6147 days
466284    7243 days
Name: earliest_cr_line_date, Length: 466285, dtype: timedelta64[ns]

In [36]:
# Assume we are now in December 2019
loan_data['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2019-12-01') - loan_data['earliest_cr_line_date']) / np.timedelta64(1, 'M')))

Some of the dates look strange with negative distances => the dataset contains dates in the future. 
The min count is -588 months. 588 / 12 = 49 years. 

It looks like the data capture does not recognise dates before 1970 = 2019 - 49.

In [37]:
loan_data['mths_since_earliest_cr_line'].describe()

count    466256.000000
mean        263.482430
std          93.974829
min        -588.000000
25%         207.000000
50%         249.000000
75%         309.000000
max         611.000000
Name: mths_since_earliest_cr_line, dtype: float64

In [38]:
loan_data.loc[: , ['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']][loan_data['mths_since_earliest_cr_line'] < 0]

,earliest_cr_line,earliest_cr_line_date,mths_since_earliest_cr_line
1580,Sep-62,2062-09-01,-513.0
1768,Sep-68,2068-09-01,-585.0
2820,Sep-64,2064-09-01,-537.0
3283,Sep-67,2067-09-01,-573.0
3360,Feb-65,2065-02-01,-542.0
...,...,...,...
464004,Jan-68,2068-01-01,-577.0
464260,Jul-66,2066-07-01,-559.0
465100,Oct-67,2067-10-01,-574.0
465500,Sep-67,2067-09-01,-573.0


In [39]:
loan_data['mths_since_earliest_cr_line'].describe()

count    466256.000000
mean        263.482430
std          93.974829
min        -588.000000
25%         207.000000
50%         249.000000
75%         309.000000
max         611.000000
Name: mths_since_earliest_cr_line, dtype: float64

Let's add 1200 months / 100 years to all observations with a negative time distance to remedy this 

In [40]:
loan_data['mths_since_earliest_cr_line'][loan_data['mths_since_earliest_cr_line'] < 0] = loan_data['mths_since_earliest_cr_line'] + 1200

C:\Users\User1\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
min(loan_data['mths_since_earliest_cr_line'])

97.0

### Time

In [42]:
loan_data['term']

0          36 months
1          60 months
2          36 months
3          36 months
4          60 months
             ...    
466280     60 months
466281     60 months
466282     60 months
466283     36 months
466284     36 months
Name: term, Length: 466285, dtype: object

In [43]:
loan_data['term'].describe()

count         466285
unique             2
top        36 months
freq          337953
Name: term, dtype: object

In [44]:
loan_data['term_int'] = loan_data['term'].str.replace(' months', '')

In [45]:
loan_data['term_int']

0          36
1          60
2          36
3          36
4          60
         ... 
466280     60
466281     60
466282     60
466283     36
466284     36
Name: term_int, Length: 466285, dtype: object

In [46]:
type(loan_data['term_int'][25])

str

In [47]:
loan_data['term_int'] = pd.to_numeric(loan_data['term'].str.replace(' months', ''))
loan_data['term_int']

0         36
1         60
2         36
3         36
4         60
          ..
466280    60
466281    60
466282    60
466283    36
466284    36
Name: term_int, Length: 466285, dtype: int64

In [48]:
type(loan_data['term_int'][0])

numpy.int64

In [49]:
loan_data['issue_d']

0         Dec-11
1         Dec-11
2         Dec-11
3         Dec-11
4         Dec-11
           ...  
466280    Jan-14
466281    Jan-14
466282    Jan-14
466283    Jan-14
466284    Jan-14
Name: issue_d, Length: 466285, dtype: object

Let's set the date as December 2017 and measure months since issue from that date.  The choice of date is not too controversial.

In [50]:
loan_data['issue_d_date'] = pd.to_datetime(loan_data['issue_d'], format = '%b-%y')
loan_data['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - loan_data['issue_d_date']) / np.timedelta64(1, 'M')))
loan_data['mths_since_issue_d'] = loan_data['mths_since_issue_d'].astype(int)
loan_data['mths_since_issue_d'].describe()


count    466285.000000
mean         51.255187
std          14.340154
min          36.000000
25%          41.000000
50%          47.000000
75%          57.000000
max         126.000000
Name: mths_since_issue_d, dtype: float64

### Preprocessing few discrete variables

In [51]:
loan_data['funded_amnt'] = loan_data['funded_amnt'].astype(int)
loan_data['funded_amnt_inv'] = loan_data['funded_amnt_inv'].astype(int)
loan_data['id'] = loan_data['id'].astype(int)
loan_data['member_id'] = loan_data['member_id'].astype(int)
loan_data['total_pymnt'] = loan_data['total_pymnt'].round(2)
loan_data['term'] = loan_data['term'].astype(str).str[0:3]

loan_data.head(2)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,vix_mean,vix_min,vix_max,vix_spread,year,ted_spread_mean,ted_spread_min,ted_spread_max,ted_spread_spread,nat_unemp,retail_sales,emp_length_int,earliest_cr_line_date,mths_since_earliest_cr_line,term_int,issue_d_date,mths_since_issue_d
0,0,1077501,1296599,5000,5000,4975,36,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,credit_card,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.07,5831.78,5000.00,861.07,0.0,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,0.0,NaN,NaN,NaN,25.05,20.73,30.59,9.86,2011-12-01,0.55,0.51,0.57,0.06,8.5,0.964808,10.0,1985-01-01,419.0,36,2011-12-01,72
1,1,1077430,1314167,2500,2500,2500,60,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,car,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.71,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,0.0,NaN,NaN,NaN,25.05,20.73,30.59,9.86,2011-12-01,0.55,0.51,0.57,0.06,8.5,0.964808,0.0,1999-04-01,248.0,60,2011-12-01,72


In [52]:
cols_to_integers = ['id','member_id', 'loan_amnt',       
                    'funded_amnt', 'funded_amnt_inv'
                  ]

for col in cols_to_integers:
    loan_data[col].astype(int)#, axis=1, inplace=True )
    
cols_to_float = ['int_rate','installment']

for col in cols_to_float:
    loan_data[col] = loan_data[col].astype('float32')

In [53]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 0 to 466284
Data columns (total 68 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Unnamed: 0                   466285 non-null  int64         
 1   id                           466285 non-null  int32         
 2   member_id                    466285 non-null  int32         
 3   loan_amnt                    466285 non-null  int64         
 4   funded_amnt                  466285 non-null  int32         
 5   funded_amnt_inv              466285 non-null  int32         
 6   term                         466285 non-null  object        
 7   int_rate                     466285 non-null  float32       
 8   installment                  466285 non-null  float32       
 9   grade                        466285 non-null  object        
 10  sub_grade                    466285 non-null  object        
 11  emp_length                

We are going to preprocess the following discrete variables: grade, sub_grade, home_ownership, verification_status, loan_status, purpose, addr_state, initial_list_status. 

Most likely, we are not going to use sub_grade, as it overlaps with grade. We will prefix the dummy variable names with discrete variable names. This will come in handy later when it comes to building the scorecards.  

In [54]:
pd.get_dummies(loan_data['grade']).head()

,A,B,C,D,E,F,G
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0


In [55]:
pd.get_dummies(loan_data['grade'], prefix = 'grade', prefix_sep = ':').head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0


Adding a naming prefix to the loan data dummy variables - this notation helps later!

In [56]:
loan_data_dummies = [pd.get_dummies(loan_data['grade'], prefix = 'grade', prefix_sep = ':'),
                     pd.get_dummies(loan_data['sub_grade'], prefix = 'sub_grade', prefix_sep = ':'),
                     pd.get_dummies(loan_data['home_ownership'], prefix = 'home_ownership', prefix_sep = ':'),
                     pd.get_dummies(loan_data['verification_status'], prefix = 'verification_status', prefix_sep = ':'),
                     pd.get_dummies(loan_data['loan_status'], prefix = 'loan_status', prefix_sep = ':'),
                     pd.get_dummies(loan_data['purpose'], prefix = 'purpose', prefix_sep = ':'),
                     pd.get_dummies(loan_data['addr_state'], prefix = 'addr_state', prefix_sep = ':'),
                     pd.get_dummies(loan_data['initial_list_status'], prefix = 'initial_list_status', prefix_sep = ':')]

Next we concatenate the dummy variables into a dataframe

In [57]:
loan_data_dummies = pd.concat(loan_data_dummies, axis = 1)
loan_data_dummies = loan_data_dummies.astype(int)

In [58]:
loan_data = pd.concat([loan_data, loan_data_dummies], axis = 1)

Let's check the columns names:

In [59]:
loan_data.columns.values

array(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim', 'vix_mean', 'vix_min', 'vix_max', 'vix_spread',
       'year

### Check for missing values and clean

In [60]:
pd.options.display.max_rows = None
loan_data.isnull().sum()

Unnamed: 0                                                              0
id                                                                      0
member_id                                                               0
loan_amnt                                                               0
funded_amnt                                                             0
funded_amnt_inv                                                         0
term                                                                    0
int_rate                                                                0
installment                                                             0
grade                                                                   0
sub_grade                                                               0
emp_length                                                          21008
home_ownership                                                          0
annual_inc                            

In [61]:
pd.options.display.max_rows = 100

To be conservative we set missing values for revolving credit limit to equal the loan amount => borrower at the limit.

In [62]:
loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'], inplace=True)

In [63]:
loan_data['total_rev_hi_lim'].isnull().sum()

0

In [64]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 0 to 466284
Columns: 194 entries, Unnamed: 0 to initial_list_status:w
dtypes: datetime64[ns](3), float32(2), float64(38), int32(131), int64(4), object(16)
memory usage: 477.1+ MB


### Treatment of N/As: 
Replace N/A s with median values where annual income. Income is heavily skewed so mean is not well suited
Replace with 0 for the remaining columns inline with a conservative risk management approach, as 0 is the poorest perceived score, or the most likely missing value as in the case inquiries in the past 6 months, where blank would mean no enquiries.

In [65]:
loan_data['annual_inc'].fillna(loan_data['annual_inc'].median(), inplace=True)
loan_data['annual_inc'] = loan_data['annual_inc'].astype('int')

In [66]:
loan_data['mths_since_earliest_cr_line'].fillna(0, inplace=True)
loan_data['acc_now_delinq'].fillna(0, inplace=True)
loan_data['total_acc'].fillna(0, inplace=True)
loan_data['pub_rec'].fillna(0, inplace=True)
loan_data['open_acc'].fillna(0, inplace=True)
loan_data['inq_last_6mths'].fillna(0, inplace=True)
loan_data['delinq_2yrs'].fillna(0, inplace=True)
loan_data['emp_length_int'].fillna(0, inplace=True)

In [67]:
loan_data.isnull().sum()

Unnamed: 0               0
id                       0
member_id                0
loan_amnt                0
funded_amnt              0
                        ..
addr_state:WI            0
addr_state:WV            0
addr_state:WY            0
initial_list_status:f    0
initial_list_status:w    0
Length: 194, dtype: int64

### Feature Transformations

Installment would be better expressed as proportion of monthly income spent on servicing the loan if originated, so we create a new feature to reflect this.

In [68]:
loan_data['payment_to_inc'] = (loan_data['installment'] * 12 *100) / (loan_data['annual_inc'])

In [69]:
loan_data['payment_to_inc'].describe()

count    466285.000000
mean          8.010283
std           4.027349
min           0.028895
25%           4.908646
50%           7.519023
75%          10.715799
max          54.171005
Name: payment_to_inc, dtype: float64

In [70]:
with open("../data/loan_data_2007_2014_preprocessed.pkl", "wb") as f:
    pickle.dump(loan_data, f)